<a href="https://colab.research.google.com/github/brayanricardo13/Analisis-Econometrico/blob/main/Cesantias_Y_pensiones_Obligatorias_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Extraer Datos**

In [1]:
# @title
# ----------------------------------------
# 1. Librerías necesarias
# ----------------------------------------
!pip install pandas requests plotly

import pandas as pd
import requests
import time
import plotly.express as px

# ----------------------------------------
# 2. Descargar datos históricos de cesantías
# ----------------------------------------

# URL del recurso JSON
base_url = "https://www.datos.gov.co/resource/uawh-cjvi.json"

# Parámetros para descarga por lotes
limit = 1000
offset = 0
all_data = []

while True:
    url = f"{base_url}?$limit={limit}&$offset={offset}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if not data:
            break
        all_data.extend(data)
        offset += limit
        time.sleep(1)  # Esperar para no saturar el servidor
    else:
        break

# Convertir datos a DataFrame
if all_data:
    df_historico = pd.DataFrame(all_data)
else:
    raise ValueError("No se descargaron datos.")

# ----------------------------------------
# 3. Preparar datos
# ----------------------------------------

# Convertir columna fecha
df_historico['fecha'] = pd.to_datetime(df_historico['fecha'])

# Ordenar
df_ordenado = df_historico.sort_values(by=['nombre_entidad', 'nombre_fondo', 'fecha'])

# ----------------------------------------
# 4. Definir el fondo que quieres analizar
# ----------------------------------------


**Seleccionar Fondo**

In [2]:
# Aquí puedes cambiar el fondo que quieres analizar:
tipo_fondo = "Fondo de Pensiones Mayor Riesgo"  # <-- Cambia aquí

In [3]:
# @title
# 5. Filtrar y procesar el fondo seleccionado
# ----------------------------------------

# Filtrar según el tipo de fondo
df_fondo = df_ordenado[df_ordenado['nombre_fondo'].str.contains(tipo_fondo, case=False)].copy()

# Asegurar tipos numéricos
df_fondo['valor_unidad'] = pd.to_numeric(df_fondo['valor_unidad'], errors='coerce')

# Encontrar primer valor de cada entidad
primer_valor = df_fondo.groupby('nombre_entidad')['valor_unidad'].first().reset_index()
primer_valor.rename(columns={'valor_unidad': 'valor_inicial'}, inplace=True)

# Unir primer valor
df_fondo = pd.merge(df_fondo, primer_valor, on='nombre_entidad')

# Calcular base 100
df_fondo['valor_base_100'] = (df_fondo['valor_unidad'] / df_fondo['valor_inicial']) * 100

# ----------------------------------------
# 6. Gráfica comparativa
# ----------------------------------------

fig = px.line(
    df_fondo,
    x='fecha',
    y='valor_base_100',
    color='nombre_entidad',
    title=f"Comportamiento de {tipo_fondo} (Base 100)",
    labels={'valor_base_100': 'Índice Base 100', 'fecha': 'Fecha', 'nombre_entidad': 'Entidad'},
    template='plotly_white'  # Fondo blanco
)

fig.show()

In [4]:
# @title
import pandas as pd
import plotly.express as px

# Suponiendo que ya tienes df_fondo como lo filtraste antes.

# Calcular el drawdown por entidad
df_fondo['max_historico'] = df_fondo.groupby('nombre_entidad')['valor_base_100'].cummax()
df_fondo['drawdown'] = (df_fondo['valor_base_100'] - df_fondo['max_historico']) / df_fondo['max_historico'] * 100

# Crear la gráfica de drawdown
fig = px.line(
    df_fondo,
    x='fecha',
    y='drawdown',
    color='nombre_entidad',
    title="Evolución del Drawdown Normalizado",
    labels={'drawdown': 'Drawdown (%)', 'fecha': 'Fecha', 'nombre_entidad': 'Entidad'},
    template='plotly_white'  # Fondo blanco
)

fig.show()


In [5]:
# @title
# 1. Calcular retornos diarios por entidad
df_fondo['retorno_diario'] = df_fondo.groupby('nombre_entidad')['valor_base_100'].pct_change()

# 2. Calcular volatilidad 90 días (desviación estándar de los retornos)
df_fondo['volatilidad_90d'] = df_fondo.groupby('nombre_entidad')['retorno_diario'].rolling(window=90).std().reset_index(0, drop=True) * (252**0.5)  # anualizado

# 3. Graficar
fig = px.line(
    df_fondo,
    x='fecha',
    y='volatilidad_90d',
    color='nombre_entidad',
    title="90 Day Rolling Volatility (Anualizada)",
    labels={'volatilidad_90d': 'Volatilidad 90d (%)', 'fecha': 'Fecha', 'nombre_entidad': 'Entidad'},
    template='plotly_white'
)

fig.show()


In [6]:
# @title
import pandas as pd
import numpy as np

# Asegurarnos que 'fecha' es tipo datetime
df_fondo['fecha'] = pd.to_datetime(df_fondo['fecha'])

# Calcular retornos diarios
df_fondo['retorno_diario'] = df_fondo.groupby('nombre_entidad')['valor_base_100'].pct_change()

# Función para calcular rentabilidad y volatilidad entre fechas
def calcular_metricas(df, fecha_corte, dias):
    fecha_inicio = fecha_corte - pd.Timedelta(days=dias)
    df_periodo = df[(df['fecha'] >= fecha_inicio) & (df['fecha'] <= fecha_corte)]

    rentabilidad = (df_periodo.groupby('nombre_entidad')['valor_base_100'].last() /
                    df_periodo.groupby('nombre_entidad')['valor_base_100'].first() - 1)

    volatilidad = df_periodo.groupby('nombre_entidad')['retorno_diario'].std() * (252**0.5)

    return rentabilidad * 100, volatilidad * 100  # en porcentaje

# Fecha de corte: último dato disponible
fecha_corte = df_fondo['fecha'].max()

# Periodos en días aproximados
periodos = {
    'Ultimo_mes': 30,
    'Ultimos_6_meses': 182,
    'Anio_Corrido': fecha_corte.timetuple().tm_yday,  # Días desde inicio de año
    'Ultimo_anio': 365,
    'Ultimos_2_anios': 730,
    'Ultimos_3_anios': 1095
}

# Diccionarios para guardar resultados
rentabilidades = {}
volatilidades = {}

# Calcular métricas
for periodo, dias in periodos.items():
    rent, vol = calcular_metricas(df_fondo, fecha_corte, dias)
    rentabilidades[periodo] = rent
    volatilidades[periodo] = vol

# Unir todo en una tabla
tabla_final = pd.concat([
    rentabilidades['Ultimo_mes'],
    rentabilidades['Ultimos_6_meses'],
    rentabilidades['Anio_Corrido'],
    rentabilidades['Ultimo_anio'],
    rentabilidades['Ultimos_2_anios'],
    rentabilidades['Ultimos_3_anios'],
    volatilidades['Ultimo_mes'],
    volatilidades['Ultimos_6_meses'],
    volatilidades['Anio_Corrido'],
    volatilidades['Ultimo_anio'],
    volatilidades['Ultimos_2_anios'],
    volatilidades['Ultimos_3_anios']
], axis=1)

# Renombrar columnas
tabla_final.columns = [
    'Rentabilidad Último mes',
    'Rentabilidad Últimos 6 meses',
    'Rentabilidad Año Corrido',
    'Rentabilidad Último año',
    'Rentabilidad Últimos 2 años',
    'Rentabilidad Últimos 3 años',
    'Volatilidad Último mes',
    'Volatilidad Últimos 6 meses',
    'Volatilidad Año Corrido',
    'Volatilidad Último año',
    'Volatilidad Últimos 2 años',
    'Volatilidad Últimos 3 años'
]

# Opcional: redondear
tabla_final = tabla_final.round(3)

# Mostrar tabla
tabla_final


,Rentabilidad Último mes,Rentabilidad Últimos 6 meses,Rentabilidad Año Corrido,Rentabilidad Último año,Rentabilidad Últimos 2 años,Rentabilidad Últimos 3 años,Volatilidad Último mes,Volatilidad Últimos 6 meses,Volatilidad Año Corrido,Volatilidad Último año,Volatilidad Últimos 2 años,Volatilidad Últimos 3 años
nombre_entidad,,,,,,,,,,,,
Colfondos S.A. Pensiones Y Cesantias,-0.175,1.833,0.956,12.622,26.447,30.187,11.229,6.057,6.970,5.322,4.921,6.085
Porvenir,-1.431,0.756,-0.277,11.830,24.894,27.505,11.570,6.290,7.266,5.441,5.048,6.242
Proteccion,-1.319,1.183,-0.167,12.020,24.321,27.176,10.611,5.866,6.682,5.246,4.925,6.097
Skandia Pensiones Y Cesantías S.A.,0.141,2.836,1.708,16.173,29.003,32.504,12.122,6.545,7.408,5.712,5.190,6.652
